# Ant Colony System: A Cooperative Learning Approach to the Traveling Salesman Problem

**Points from the Papers**   
- Ant Colony System (ACS) Distributed Algorithm applied to TSP.   
- In ACS, a set of cooperating agents cooperate to find good solutions to TSPs.   
- ACS outperforms other nature-inspired Algorithms such as simulated Annealing and evolutionary computation.   
    
**LOGIC**   
- Real ants are capable of finding the shortest path from a food source to their nest without using visual cues.   
- While walking, ants deposit pheromone on the ground and follow, in probability, pheromone deposited by other ants.   

![How real Ants find the Shortest path](diagrams/img1.jpg)

(a) Ants arrive at a decision point (marked by ?)  
   
(b) Some ants choose the upper path and some the lower path. The choice is random.   
   
(c) Since ants move at approximately a constant speed, the ants which choose the lower, shorter path reach the opposite decision point faster than those which choose the upper longer path.   
   
(d) Pheromone accumulates a higher rate on the shorter path. The number of dashed lines is approximately proportional to the amount of pheromone deposited by ants.   

### TSP Algorithm   
Let V = {a, ..., z} be a set of Cities.   
A = {(r, s): r, s belong to V} be edge Set.   
Delta(r, s) = Delta(s, r) be a cost measured associated with edge (r,s) belong to A.   
   
The TSP Problem is the problem of finding a minimal cost closed tour that visits each city once. In the case cities r belong to V are given by their coordinates (x, y) and Delta(r, s) is the Eucledian Distance between r and s. Then we have a Eucledian TSP.   

**Background**   
- Ant System utilizes a graph Representation.   
- In addition to the cost measure Delta(r, s), each edge (r,s) has also a desirability measure Tao(r, s) called Pheromone, which is updated at run time by artificial ants.    
   
Each Any generates a complete tour by choosing the cities according to a probabilistic state Transition rule; ants prefer to move to cities which are connected by short edges with a high amount of pheromone.    
   
Once all ants have completed their tours a global pheromone updating rule is applied, a fraction of the phermone evaporates on all edges (Hence, edges that are not refreshed become less desired), and then each ant deposits an amount of pheromone on edges which belong to its tour in proportion to how short its tour was (in other words Edges that belong to many short tours are the edges which recive the greater amount of pheromone.) The process is then iterated.  

### ACS Algorithm  
![ACS Algorithm](diagrams/img6.jpg)

**State Transition Rule**   
![State Transition Rule](diagrams/img5.jpg)

Probability with which ant **k** chooses to move to the city s.Mutiply the Pheromone on edge (r, s) by the corresponding heuristic value. In this way, we favor the choice of edges which are shorter and which have greater amount of Pheromone.   

**Global Updating Rule**   
![Global Updating Rule](diagrams/img4.jpg)

Alpha is a pheromone decay Parameter.   
Lk is the length of the tour performed by ant k, and m is the number of ants.   
Pheromone updating is intended to allocate a greater amount of pheromone to shorter tours. In a sense, this is similar to a reinforcement learning scheme.   
   
Pheromone deposited on the edges plays the role of a distributed long-term memory; this memory is not stored locally within the individual ants, but is distributed on the edges of the graph. This allows an indirect form of communication called stigmergy.   

**Though Ant System was useful for discovering good or optimal solutions for small TSPs, the time required to find such results made it infeasible for larger problems. Three main changes to improve its performance which lead to the definition of the ACS.**

**Three New Aspects of the ACS Algorithm** . 
1. The State Transition rule provides the direct way to balance between exploration of new edges and exploitation of a priori and accumulated knowledge about the problem   
2. The Global updating rule is applied only to edges which belong to the best any tour   
3. While ants construct a solution a local pheromone updating rule is appled. 

Informally,  
- m ants are initially positioned on n cities chosen according to some initialization rule (random) .  
- Each ants builds a tour by repeatedly applying a stochastic greedy rule. While constructing its tour, an ant also modifies the amount of pheromone on the visited edges by applying the local updating rule.   
- Once all ants have terminated the tour, the amount of pheromone on edges is modified again.   
- An edge with the higher amount of pheromone is a desired choice. 

**New State Transition Rule**   
![State Transition Rule](diagrams/img3.jpg)

where q is the random no uniformly distributed in [0, 1].   
q0 is a parameter (0 < qo < 1) and S is a random variable.   
   
Every time an ant in city r has to choose a city s to move to, it samples a random number 0 <= q <= 1,    
If q <= q0, then the best edge is chosen (exploitation) .  
Else an edge is choosen according to (1) (Biased exploration) .

**New Global Updating Rule**   
![Global Updation Rule](diagrams/img2.jpg)

Only those edges belonging to the globally best tour will recieve reinforement. 

**New Local Updating Rule**   
![Local Updation Rule](diagrams/img7.jpg)

## Code

In [8]:
import random as rn
import numpy as np
from numpy.random import choice as np_choice
import random
import pandas as pd
import operator
import time

In [15]:
class AntColony:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        """
        Args:
            distances (2D numpy.array): Square matrix of distances. Diagonal is assumed to be np.inf.
            n_ants (int): Number of ants running per iteration
            n_best (int): Number of best ants who deposit pheromone
            n_iteration (int): Number of iterations
            decay (float): Rate it which pheromone decays. The pheromone value is multiplied by decay, so 0.95 will lead to decay, 0.5 to much faster decay.
            alpha (int or float): exponenet on pheromone, higher alpha gives pheromone more weight. Default=1
            beta (int or float): exponent on distance, higher beta give distance more weight. Default=1

        Example:
            ant_colony = AntColony(german_distances, 100, 20, 2000, 0.95, alpha=1, beta=2)          
        """
        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

        
    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            #print("Length of the Shortest path during "+str(i)+" iteration is "+str(shortest_path[1]))
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path            
            self.pheromone * self.decay            
        return all_time_shortest_path

    def spread_pheronome(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start)) # going back to where we started    
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)

        norm_row = row / row.sum()
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move


In [16]:
def read_data(count):
        df = pd.read_csv('../data.csv', header=None) 
        nodes = []
        for i in range(len(df[0])):
            sp = df[0][i].split(' ')
            x = float(sp[1])
            y = float(sp[2])
            nodes.append([x, y])
        nodes = nodes[:count]   
        return nodes
    
    
def eucledian_distance(a, b):
    ret = 0
    for i in range(len(a)):
        ret += (a[i] - b[i]) ** 2
    return ret ** 0.5

In [20]:
start = time.time()

beta  = 2
q0 = 0.95
alpha = 0.1
peta = 0.1
n_ants = 20 #No of Ants
m_ants = 5
iterations = 100

nodes = read_data(500)

arrs = [[np.inf]*len(nodes)]*len(nodes)

for i in range(len(nodes)):
    for j in range(i+1, len(nodes)):
        arrs[i][j] = eucledian_distance(nodes[i], nodes[j])
        arrs[j][i] = arrs[i][j]
        
distances = np.array(arrs)

ant_colony = AntColony(distances, n_ants, m_ants, iterations, q0, alpha, beta)

shortest_path = ant_colony.run()
print('Length of the Shortest path', str(shortest_path[1]))

time_lapse = time.time() - start
print('Time taken to find shortest path', time_lapse)

Length of the Shortest path 568805.7504141958
Time taken to find shortest path 163.5919485092163


**REFERENCES**   
1. https://github.com/Akavall/AntColonyOptimization
2. http://people.idsia.ch/~luca/acs-ec97.pdf   
3. https://en.wikipedia.org/wiki/Ant_colony_optimization_algorithms